In [2]:
import numpy as np
import pandas as pd
import math
import xlrd

In [3]:
holiday = [
    "2019-01-01",
    "2019-12-25"
]

In [4]:
holiday = [pd.Timestamp(int(date.split("-")[0]), int(date.split("-")[1]), int(date.split("-")[2])) for date in holiday]

In [5]:
def map_step_to_station(df):
    output = {}
    for index, step in enumerate(df.iloc[:, 0]):
        stations = [df.iloc[index, i] for i in range(1,df.shape[1]) if type(df.iloc[index, i]) == str]
        output[step] = stations
    return output

In [6]:
class STEP:
    def __init__(self, UPH=0, num_unit=0):
        self.UPH = UPH
        self.num_unit = num_unit
        
    def return_time_required(self):
        return self.UPH * self.num_unit

In [7]:
class STATION:
    def __init__(self, working_hours=8.5, utilization=1, required_hours=0, num_labor=1):
        self.working_hours = working_hours
        self.required_hours = required_hours
        self.utilization = utilization
        self.num_labor = num_labor
        
    def total_working_hours(self):
        return self.working_hours*self.num_labor*self.utilization
    
    def return_utlization(self):
        if required_hours > working_hours*num_labor*utilization:
            self.utilization = self.required_hours/(self.working_hours*self.num_labor*self.utilization)
        return self.utilization

In [8]:
steps_to_stations = pd.ExcelFile("excels/steps_to_station.xlsx")
steps_to_station = steps_to_stations.parse(steps_to_stations.sheet_names[0])
steps_to_stations = map_step_to_station(steps_to_station)

productType_to_outputPoint = pd.ExcelFile("excels/productType_to_outputPoint.xlsx")
productType_to_outputPoint = productType_to_outputPoint.parse(productType_to_outputPoint.sheet_names[0])
productType_to_outputPoint = {productType_to_outputPoint.iloc[index, 0]: productType_to_outputPoint.iloc[index, 1]
                              for index in range(productType_to_outputPoint.shape[0])}

demand = pd.ExcelFile("excels/demand.xlsx")
demand = demand.parse(demand.sheet_names[0])
# sort demand by deadline time
index = pd.to_datetime(demand["deadline"], format="%Y%m%d:%H:%M:%S.%f").sort_values().index
demand = demand.iloc[index, :]

labor_arrangement = pd.ExcelFile("excels/labor_arrangement.xlsx")
labor_arrangement = labor_arrangement.parse(labor_arrangement.sheet_names[0])
working_hours = {labor_arrangement.iloc[index, 0]: labor_arrangement.iloc[index, 2] 
                     for index in range(labor_arrangement.shape[0])}
labor_arrangement = {labor_arrangement.iloc[index, 0]: labor_arrangement.iloc[index, 1] 
                     for index in range(labor_arrangement.shape[0])}

UPH = pd.ExcelFile("excels/UPH.xlsx")
UPH = UPH.parse(UPH.sheet_names[0])
step_UPH = {UPH.iloc[index, 0]: UPH.iloc[index, 1] for index in range(UPH.shape[0])}

In [9]:
# setting up
steps = ['Wafer to Cell', 'Cell to Bar', 'Facet Visual Inspection', 'HR/AR Coating', 'Bar to Die Visual Inspection', 
         'Bar To Die ', 'Bar Test LIV', 'Visual Inpection', 'Die Pick', 'Facet Visual Inspection', 'Die Attach (Die and TM5)', 
         'Wirebond', 'Pre Screen LIV', 'Pre Screen OSA', 'Visual Inpection', 'Burn In Test', 'Post Screen LIV', 
         'Post Screen OSA', 'Visual Inspection', 'OGR/ATR.', 'Shipment to Outsource', 'Post Screen LIV', 'Post Screen OSA',
         'Burn In Test', 'Post Screen LIV', 'Post Screen OSA', 'OGR/ATR.', 'Shipment of Module']

# need to be modified if the steps are changed
outputPoint_to_steps = {"Module": -1, 'Cos': -8, "Die": 9}

# define stations, steps and corresponding objects
stations = list(set(list(steps_to_station['Variant 1']) + list(steps_to_station['Variant 2']) 
                    + list(steps_to_station['Variant 3'])) - set([np.nan]))
steps = list(steps_to_station['Process'])
step_required_time = {step: 0 for step in steps}
stations_objects = {station: STATION(working_hours=working_hours[station], num_labor=labor_arrangement[station]) 
                    for station in stations}

# group steps which use the same stations
number_stations_group = {0: []}
number_to_steps = {}
count = 0
for step in steps_to_stations:
    if steps_to_stations[step] not in list(number_stations_group.values()):
        count+=1
        number_stations_group[count] = steps_to_stations[step]
del number_stations_group[0]

for number in number_stations_group:
    steps_number = [step for step in steps_to_stations if steps_to_stations[step] == number_stations_group[number]]
    number_to_steps[number] = steps_number

In [10]:
def to_date(timestamp):
    return "{}-{}-{}".format(timestamp.year, timestamp.month, timestamp.day)

In [11]:
def main(demand, steps_list, stations_objects, step_required_time, sort=False):
    overload = {}
    
    for idx, order in enumerate(demand.iloc[:, 0]):
        output_point = productType_to_outputPoint[order]
        steps = steps_list[:outputPoint_to_steps[output_point]]
        for step in steps:
            step_object = STEP(step_UPH[step], demand.iloc[idx, 1])
            step_required_time[step] += step_object.return_time_required()
            
        date_list = list(pd.date_range(start=pd.Timestamp.today().date(), end=demand['deadline'][idx].date()))
        date_list = list(set(date_list) - set(holiday))
        weekday_list = [day for day in date_list[1:] if day.weekday() < 5]
        
        addition_time = 0
        for number in number_to_steps:
            steps = number_to_steps[number]
            stations = number_stations_group[number]
            required_time = 0
            
            # special number
            if number==5:
                for step in steps:
                    required_time_5 = step_required_time[step]
                    station_object = stations_objects['Auto Bar Tester']
                    available_time = station_object.total_working_hours() - station_object.required_hours
                    if required_time_5 < available_time: 
                        station_object.required_hours = required_time_5
                        required_time_5
                    else:
                        required_time_5 -= available_time
                        station_object.required_hours = station_object.total_working_hours()
                    addition_time += required_time_5
            
            else:
                # sum up required time of similar steps
                for step in steps:
                    required_time += step_required_time[step]
                if number==9:
                    required_time += addition_time
                    
                daily_station_required_time = required_time/len(date_list)/len(stations)
                # sum up available time
                for station in stations:
                    station_object = stations_objects[station]
                    available_time = station_object.total_working_hours()
                    overload_addition = pd.DataFrame()
                    utilization = daily_station_required_time/available_time
                    suggestion = "increase working time to {}".format(utilization*station_object.working_hours)
                    if utilization > 1:
                        for date in date_list:
                            date = to_date(date)
                            if (station, date) not in overload: 
                                overload[(station, date)] = [utilization ,suggestion]
                            else:
                                if overload[(station, date)][0] < utilization: 
                                    overload[(station, date)] = [utilization ,suggestion]
        
        station_to_uti = {}
        for key in overload:
            one = [key[1], overload[key][0], overload[key][1]]
            if key[0] in station_to_uti:
                station_to_uti[key[0]].append(one)
            else:
                station_to_uti[key[0]] = [one]
        output = pd.DataFrame()
        for station in station_to_uti:
            lst = np.array(station_to_uti[station])
            station_df = pd.DataFrame()
            station_df['Station'] = [station] * lst.shape[0]
            station_df['Date'] = lst[:, 0]
            station_df['Utilization'] = lst[:, 1]
            station_df['Suggestion'] = lst[:, 2]
            index = station_df['Date'].sort_values().index
            station_df = station_df.iloc[index, :]
            output = pd.concat([output, station_df])
            
        if sort:
            index = output['Date'].sort_values().index
            output = output.iloc[index, :]
        return output

In [12]:
def process():
    step_required_time = {step: 0 for step in steps}
    df = main(demand, steps, stations_objects, step_required_time)
    return df

In [13]:
df = process()

In [14]:
df

,Station,Date,Utilization,Suggestion
47,Wirebonder,2019-4-26,1.2254901960784312,increase working time to 10.416666666666666
33,Wirebonder,2019-4-27,1.2254901960784312,increase working time to 10.416666666666666
11,Wirebonder,2019-4-28,1.2254901960784312,increase working time to 10.416666666666666
2,Wirebonder,2019-4-29,1.2254901960784312,increase working time to 10.416666666666666
12,Wirebonder,2019-4-30,1.2254901960784312,increase working time to 10.416666666666666
27,Wirebonder,2019-5-1,1.2254901960784312,increase working time to 10.416666666666666
23,Wirebonder,2019-5-10,1.2254901960784312,increase working time to 10.416666666666666
3,Wirebonder,2019-5-11,1.2254901960784312,increase working time to 10.416666666666666
21,Wirebonder,2019-5-12,1.2254901960784312,increase working time to 10.416666666666666
0,Wirebonder,2019-5-13,1.2254901960784312,increase working time to 10.416666666666666
